In [ ]:
%matplotlib nbagg

import numpy as np
import numpy.linalg as lin

from IPython.display import display,HTML,clear_output
import time

import cv2

import matplotlib as mpl
import matplotlib.pyplot as plt

import math
from math import pi as PI

mpl.use('nbagg')

from matplotlib import animation
mpl.rc('animation', html='html5') #display animated plots inline

from robmob.robot import Robot
from robmob.sensors import KinectRGBSensor
from robmob.visualization import Visualizer

In [ ]:
ip_robot = '192.168.0.109'
robot = Robot(ip_robot)
robot.connect()

In [ ]:
kinect = KinectRGBSensor()
robot.add_sensor(kinect)
img1 = cv2.imread('qr.jpg',0)          # queryImage
# print("type of img1: ", type(img1))
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
print(kp1[0])

In [ ]:
def get_inlines(image):
    frame = np.array(image)
#     print(frame)
    
    MIN_MATCH_COUNT = 10

    kp2, des2 = sift.detectAndCompute(frame,None)
    
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good) > MIN_MATCH_COUNT:
        return len(good)
    else:
        return 0

In [ ]:


def scan_inlines(angle, speed):
    max_inlines = False
    inlines = 0
    val=[]
    while not max_inlines:
        robot.angular_movement(angle, speed)
#         clear_output(wait=True)
        image = kinect.peek_data()
        display(image)
        new_inlines = get_inlines(image)
        print("new_ilneles " , new_inlines, "inlines: ", inlines)
        time.sleep(1)
        kinect.buffer.clear()
        clear_output(wait=True)
        
        if inlines <= new_inlines:
            inlines = new_inlines
            print("not max_inlines")
        else:
            max_inlines = False
            print("yes")
            return
            

In [ ]:
# liste = scan_inlines()
# rotate_to_max(liste)
scan_inlines(PI/8,1)
scan_inlines(-PI/16,0.5)

In [ ]:
kinect.buffer.clear()

In [ ]:
from IPython.display import display,HTML,clear_output
import time

N_OF_FRAMES_TO_DISPLAY = 100

for i in range(N_OF_FRAMES_TO_DISPLAY):
    clear_output(wait=True)
    display(kinect.peek_data())
    time.sleep(0.09999)